### Building polymers with the supramolecular tooklit (stk)

In [6]:
import stk
import rdkit, rdkit.Chem as rdkit

In [7]:
a = rdkit.MolFromSmiles('Brc1ccc(Br)cc1')
rdkit.AddHs(a)
rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

b = rdkit.MolFromSmiles('Brc1ccc(Br)nc1')
rdkit.AddHs(b)
rdkit.AllChem.EmbedMolecule(b, rdkit.AllChem.ETKDG())

A = stk.StructUnit2.rdkit_init(a, 'bromine')
B = stk.StructUnit2.rdkit_init(b, 'bromine')

polymer = stk.Polymer([A, B], stk.Linear("AB", [0, 0], n=8))

stk.rdkit_ETKDG(polymer)

polymer.write('isabelle-test.mol')

### Now for a homopolymer

In [26]:
a = rdkit.MolFromSmiles('Brc1cnc(Br)cn1')
rdkit.AddHs(a)
rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

A = stk.StructUnit2.rdkit_init(a, 'bromine')

polymer = stk.Polymer([A], stk.Linear("A", [0], n=8))

stk.rdkit_ETKDG(polymer)

polymer.write('isabelle-test.mol')

### Performing a conformer search

In [30]:
def conformer_search(mol, nconfs):
    
    confs = rdkit.AllChem.EmbedMultipleConfs(mol, nconfs, rdkit.AllChem.ETKDG())
    rdkit.SanitizeMol(mol)    
    
    lowest_energy = 1000000000
    for conf in confs:
        ff = rdkit.AllChem.MMFFGetMoleculeForceField(mol, rdkit.AllChem.MMFFGetMoleculeProperties(mol), confId=conf)
        ff.Initialize()
        energy = ff.CalcEnergy()
    
        if energy < lowest_energy:
            lowest_energy = energy
            lowest_conf = conf
            print('conformer:', conf, 'energy:', energy)
    
    rdkit.MolToMolFile(mol, 'conformer-search-output.mol', confId=lowest_conf)

In [33]:
conformer_search(polymer.mol, 100)

conformer: 0 energy: 1099.7067183249192
conformer: 1 energy: 682.2948909587814
conformer: 2 energy: 671.9964287187922
conformer: 5 energy: 671.8715171388501
conformer: 7 energy: 667.6413661864127
conformer: 16 energy: 637.9260740714606
conformer: 61 energy: 637.1064687072724


### Wrapping polymer building into a function

In [40]:
def build_polymer(smiles1, smiles2):
    
    a = rdkit.MolFromSmiles(smiles1)
    rdkit.AddHs(a)
    rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

    b = rdkit.MolFromSmiles(smiles2)
    rdkit.AddHs(b)
    rdkit.AllChem.EmbedMolecule(b, rdkit.AllChem.ETKDG())

    A = stk.StructUnit2.rdkit_init(a, 'bromine')
    B = stk.StructUnit2.rdkit_init(b, 'bromine')

    polymer = stk.Polymer([A, B], stk.Linear("AB", [0, 0], n=4))

    stk.rdkit_ETKDG(polymer)

    polymer.write('isabelle-test.mol')
    
    return polymer

In [41]:
build_polymer('Brc1ccc(Br)cc1', 'Brc1ccc(Br)cc1')

Polymer(building_blocks=["StructUnit2 ['bromine', 'InChI=1S/C6H4Br2/c7-5-1-2-6(8)4-3-5/h1-4H']", "StructUnit2 ['bromine', 'InChI=1S/C6H4Br2/c7-5-1-2-6(8)4-3-5/h1-4H']"], topology=Linear(ends='h', n=4, orientation=[0, 0], repeating_unit='AB'))